## Càrrega de llibreries

In [ ]:
#Llibreries requerides
import os
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img, ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet50_in_prep
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from tensorflow.keras.callbacks  import EarlyStopping, ModelCheckpoint, Callback
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
# Display
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.image as mpimg

In [ ]:
#Llibreria adicional pel disposar de models basats en arquitectures Squeeze-and-excitation
!pip install git+https://github.com/qubvel/classification_models.git

from classification_models.keras import Classifiers

## Definició de constants

In [ ]:
#Tamany de la imatge a llegir
IM_WIDTH = 224
IM_HEIGHT = 224
SIZE = (IM_WIDTH,IM_HEIGHT)
#Etiquetes
NUM_CLASSES = 3 # 0-negative, 1-neutral, 2-positive
CLASSES_NAMES = ['Negatiu','Neutre','Positiu']

In [ ]:
#Directoris de treball
TWITTER_DS_DIR = '/kaggle/input/twittertestdataset/'
BT4SA_DS_DIR = '/kaggle/input/iteracio3/iteracio3/'
RESULTATS_DIR = '/kaggle/input/tfmtestresults/'

## Funcions

### Creació de models

In [ ]:
#Funció build_ResNet50 que retorna el model base basat en ResNet50
def build_ResNet50(weights=None):
    resnet_pretrained_model = ResNet50(include_top = False, #les capes fully-connected les afegirem segons la nostra necessitat
                                       weights = weights, #Model ResNet50 pre-entrenat amb ImageNet o None
                                       pooling = 'avg',
                                       input_tensor=Input(shape=(IM_WIDTH, IM_HEIGHT,3)))
    #Evitem entrenar les capes del model resnet50 pre-entrenat
    for layer in resnet_pretrained_model.layers:
        layer.trainable = False
    return resnet_pretrained_model

def build_SEResNet50(weights=None):
    
    SEResNet50, seresnet50_in_prep = Classifiers.get('seresnet50')
    seresnet_model = SEResNet50(input_shape=(IM_WIDTH,IM_HEIGHT,3), weights=weights, include_top=False)
    
    for layer in seresnet_model.layers:
        layer.trainable = False
   
    return seresnet_model,seresnet50_in_prep

def create_model(pretrained_model, weights, add_pooling=False):
    #creació del model final basat en un pre-entrenat
    modelx = pretrained_model.output
    if add_pooling == True:
        modelx = GlobalAveragePooling2D()(modelx)
    modelx = Dense(1024, activation='relu')(modelx)
    modelx = Dropout(0.4)(modelx)
    modelx = Dense(512, activation='relu')(modelx)
    modelx = Dropout(0.4)(modelx)
    output = Dense(NUM_CLASSES, activation='softmax')(modelx)
    return_model = Model(inputs=pretrained_model.inputs, outputs=output)
    #Càrrega dels pesos passats com a paràmetre
    if weights != None:
        return_model.load_weights(weights)
    
    return return_model


### Avaluació de models

In [ ]:
def plot_confusion_matrix(y_test_gt, y_test_pred, plot_title):
    #Generació de la matriu de confusió
    cm = confusion_matrix(y_test_gt, y_test_pred)
    df_cm = pd.DataFrame(cm, index = ("Negatiu", "Neutre", "Positiu"),
                           columns = ("Negatiu", "Neutre", "Positiu"))
    #Visualització
    ax = plt.axes()
    sns.heatmap(df_cm, annot=True, cmap='Blues', fmt='d', ax=ax);
    ax.set_title(plot_title)
    ax.set(xlabel='Predicted', ylabel='True');

## Test del model basat en ResNet50

### Càrrega de dades

In [ ]:
#1 - Validació del model amb el subconjunt de validació del dataset BT4SA_Subset
test_datagen = ImageDataGenerator(preprocessing_function=resnet50_in_prep)

test_generator = test_datagen.flow_from_directory(
  BT4SA_DS_DIR + 'test/',
  target_size=(IM_WIDTH, IM_HEIGHT),
  shuffle=False,
  batch_size=32,
  seed=42,
  class_mode='categorical',  
)
y_test_true = test_generator.classes

#2 - Validació del model amb el conjunt de 'Twitter Test Dataset'
test_generator_tds = test_datagen.flow_from_directory(
 TWITTER_DS_DIR,
  target_size=(IM_WIDTH, IM_HEIGHT),
  shuffle=False,
  batch_size=32,
  seed=42,
  class_mode='categorical',
  classes = ['negatiu','neutre','positiu']
)

y_tds_test_true = test_generator_tds.classes

### Generació del model

Construim el model base 'ResNet50' sobre el que afegirem les capes pròpies i carregarem els pesos dels entrenaments realitzats

In [ ]:
modelx = build_ResNet50()

### Optimitzador Adam

Provem el model generat basat en ResNet50 i compilat fent servir l'optimitzador 'Adam'

In [ ]:
resnet50_model1 = create_model(modelx, RESULTATS_DIR + 'resnet50_weights_adam.hdf5')
resnet50_model1.compile(optimizer=tf.keras.optimizers.Adam(),
                        loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#Visualització Model
plot_model(resnet50_model1, show_shapes=True, to_file="resnet50.png")
plot_model(resnet50_model1, show_shapes=True)

#### Mètriques sobre BT4SA_Subset

In [ ]:
#1 - Avaluació del model
resnet50_model1.evaluate(test_generator)

In [ ]:
#2 - Generació de prediccions
y_test_pred = resnet50_model1.predict(test_generator)

In [ ]:
#3 - Classification report
y_test_pred_class = np.argmax(y_test_pred,axis=1)
class_report_resnet50 = classification_report(y_test_true, y_test_pred_class)
print(class_report_resnet50)

In [ ]:
#4 - Matriu de confusió
plot_confusion_matrix(y_test_true, y_test_pred_class, 'Model ResNet50 (Adam) - BT4SA Dataset')

#### Mètriques sobre Twitter_Dataset

In [ ]:
#1 - Generació de prediccions
y_test_pred_tds = resnet50_model1.predict(test_generator_tds)

In [ ]:
#2 - Classification report
y_test_pred_class_tds = np.argmax(y_test_pred_tds,axis=1)
#zero_division=1- el DS no te exemples de casos 'neutre'
class_report_resnet50 = classification_report(y_tds_test_true, y_test_pred_class_tds, zero_division=1)
print(class_report_resnet50)

In [ ]:
#3 - Matriu de confusió
plot_confusion_matrix(y_tds_test_true, y_test_pred_class_tds, 'Model ResNet50 (Adam) - Twitter Dataset')

### Optimitzador SGD

Provem el model generat basat en ResNet50 i compilat fent servir l'optimitzador 'SGD' amb moment Nesterov

In [ ]:
resnet50_model2 = create_model(modelx, RESULTATS_DIR + 'resnet50_CP_weights_SGD_NAG.hdf5')
resnet50_model2.compile(optimizer=tf.keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True),
                        loss='categorical_crossentropy', metrics=['accuracy'])

#### Mètriques sobre BT4SA_Subset

In [ ]:
#1 - Avaluació del model
resnet50_model2.evaluate(test_generator)


In [ ]:
#2 - Generació de prediccions
y_test_pred = resnet50_model2.predict(test_generator)

In [ ]:
#3 - Classification report
y_test_pred_class = np.argmax(y_test_pred,axis=1)
class_report_resnet50 = classification_report(y_test_true, y_test_pred_class)
print(class_report_resnet50)

In [ ]:
#4 - Matriu de confusió
plot_confusion_matrix(y_test_true, y_test_pred_class, 'Model ResNet50 (SGD) - BT4SA Dataset')

#### Mètriques sobre Twitter_Dataset

In [ ]:
#1 - Generació de prediccions
y_test_pred_tds = resnet50_model2.predict(test_generator_tds)

In [ ]:
#2 - Classification report
y_test_pred_class_tds = np.argmax(y_test_pred_tds,axis=1)
#zero_division=1- el DS no te exemples de casos 'neutre'
class_report_resnet50 = classification_report(y_tds_test_true, y_test_pred_class_tds, zero_division=1)
print(class_report_resnet50)

In [ ]:
#3 - Matriu de confusió
plot_confusion_matrix(y_tds_test_true, y_test_pred_class_tds, 'Model ResNet50 (SGD) - Twitter Dataset')

### Optimitzador Nadam

Provem el model generat basat en ResNet50 i compilat fent servir l'optimitzador 'Nadam', una variació de l'optimitzador Adam que utiltiza el moment Nesterov

In [ ]:
resnet50_model3 = create_model(modelx, RESULTATS_DIR + 'resnet50_CP_weights_nadam.hdf5')
resnet50_model3.compile(optimizer=tf.keras.optimizers.Nadam(),
                        loss='categorical_crossentropy', metrics=['accuracy'])

#### Mètriques sobre BT4SA_Subset

In [ ]:
#1 - Avaluació del model
resnet50_model3.evaluate(test_generator)


In [ ]:
#2 - Generació de prediccions
y_test_pred = resnet50_model3.predict(test_generator)

In [ ]:
#3 - Classification report
y_test_pred_class = np.argmax(y_test_pred,axis=1)
class_report_resnet50 = classification_report(y_test_true, y_test_pred_class)
print(class_report_resnet50)

In [ ]:
#4 - Matriu de confusió
plot_confusion_matrix(y_test_true, y_test_pred_class, 'Model ResNet50 (Nadam) - BT4SA Dataset')

#### Mètriques sobre Twitter_Dataset

In [ ]:
#1 - Generació de prediccions
y_test_pred_tds = resnet50_model3.predict(test_generator_tds)

In [ ]:
#2 - Classification report
y_test_pred_class_tds = np.argmax(y_test_pred_tds,axis=1)
#zero_division=1- el DS no te exemples de casos 'neutre'
class_report_resnet50 = classification_report(y_tds_test_true, y_test_pred_class_tds, zero_division=1)
print(class_report_resnet50)

In [ ]:
#3 - Matriu de confusió
plot_confusion_matrix(y_tds_test_true, y_test_pred_class_tds, 'Model ResNet50 (Nadam) - Twitter Dataset')

## Test del model basat en SEResNet50

### Generació del model 

In [ ]:
modelx, seresnet50_in_prep = build_SEResNet50()

### Càrrega de dades

In [ ]:
#1 - Validació del model amb el subconjunt de validació del dataset BT4SA_Subset
test_datagen = ImageDataGenerator(preprocessing_function=seresnet50_in_prep)

test_generator = test_datagen.flow_from_directory(
  BT4SA_DS_DIR + 'test/',
  target_size=(IM_WIDTH, IM_HEIGHT),
  shuffle=False,
  batch_size=32,
  seed=42,
  class_mode='categorical',  
)
y_test_true = test_generator.classes

#2 - Validació del model amb el conjunt de 'Twitter Test Dataset'
test_generator_tds = test_datagen.flow_from_directory(
 TWITTER_DS_DIR,
  target_size=(IM_WIDTH, IM_HEIGHT),
  shuffle=False,
  batch_size=32,
  seed=42,
  class_mode='categorical',
  classes = ['negatiu','neutre','positiu']
)

y_tds_test_true = test_generator_tds.classes

### Optimitzador Adam

Provem el model generat basat en ResNet50 i compilat fent servir l'optimitzador 'Adam'

In [ ]:
SEResnet50_model1 = create_model(modelx,  RESULTATS_DIR + 'seresnet50_weights_adam.hdf5' , add_pooling=True)
SEResnet50_model1.compile(optimizer=tf.keras.optimizers.Adam(),
                         loss='categorical_crossentropy', metrics=['accuracy'])

#Visualització Model
plot_model(SEResnet50_model1, show_shapes=True, to_file="seresnet50.png")
plot_model(SEResnet50_model1, show_shapes=True)

#### Mètriques sobre BT4SA_Subset

In [ ]:
#1 - Avaluació del model
SEResnet50_model1.evaluate(test_generator)

In [ ]:
#2 - Generació de prediccions
y_test_pred = SEResnet50_model1.predict(test_generator)

In [ ]:
#3 - Classification report
y_test_pred_class = np.argmax(y_test_pred,axis=1)
class_report_resnet50 = classification_report(y_test_true, y_test_pred_class)
print(class_report_resnet50)

In [ ]:
#4 - Matriu de confusió
plot_confusion_matrix(y_test_true, y_test_pred_class, 'Model SE-ResNet-50 (Adam) - BT4SA Dataset')

#### Mètriques sobre Twitter_Dataset

In [ ]:
#1 - Generació de prediccions
y_test_pred_tds = SEResnet50_model1.predict(test_generator_tds)

In [ ]:
#2 - Classification report
y_test_pred_class_tds = np.argmax(y_test_pred_tds,axis=1)
#zero_division=1- el DS no te exemples de casos 'neutre'
class_report_seresnet50 = classification_report(y_tds_test_true, y_test_pred_class_tds, zero_division=1)
print(class_report_seresnet50)

In [ ]:
#3 - Matriu de confusió
plot_confusion_matrix(y_tds_test_true, y_test_pred_class_tds, 'Model SE-ResNet-50 (Adam) - Twitter Dataset')

### Optimitzador SGD

Provem el model generat basat en SE-ResNet-50 i compilat fent servir l'optimitzador 'SGD' amb moment Nesterov

In [ ]:
SEResnet50_model2 = create_model(modelx,  RESULTATS_DIR + 'seresnet50_CP_weights_SGD_NAG.hdf5' , add_pooling=True)
SEResnet50_model2.compile(optimizer=tf.keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True),
                         loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
plot_model(SEResnet50_model2, show_shapes=True)

#### Mètriques sobre BT4SA_Subset

In [ ]:
#1 - Avaluació del model
SEResnet50_model2.evaluate(test_generator)

In [ ]:
#2 - Generació de prediccions
y_test_pred = SEResnet50_model2.predict(test_generator)
#3 - Classification report
y_test_pred_class = np.argmax(y_test_pred,axis=1)
class_report_resnet50 = classification_report(y_test_true, y_test_pred_class)
print(class_report_resnet50)

In [ ]:
#4 - Matriu de confusió
plot_confusion_matrix(y_test_true, y_test_pred_class, 'Model SE-ResNet-50 (SGD) - BT4SA Dataset')

#### Mètriques sobre Twitter_Dataset

In [ ]:
#1 - Generació de prediccions
y_test_pred_tds = SEResnet50_model2.predict(test_generator_tds)
#2 - Classification report
y_test_pred_class_tds = np.argmax(y_test_pred_tds,axis=1)
#zero_division=1- el DS no te exemples de casos 'neutre'
class_report_seresnet50 = classification_report(y_tds_test_true, y_test_pred_class_tds, zero_division=1)
print(class_report_seresnet50)

In [ ]:
#3 - Matriu de confusió
plot_confusion_matrix(y_tds_test_true, y_test_pred_class_tds, 'Model SE-ResNet-50 (SGD) - Twitter Dataset')

### Optimitzador Nadam

Provem el model generat basat en ResNet50 i compilat fent servir l'optimitzador 'Nadam', una variació de l'optimitzador Adam que utiltiza el moment Nesterov

In [ ]:
SEResnet50_model3 = create_model(modelx,  RESULTATS_DIR + 'seresnet50_CP_weights_nadam.hdf5' , add_pooling=True)
SEResnet50_model3.compile(optimizer=tf.keras.optimizers.Nadam(),
                         loss='categorical_crossentropy', metrics=['accuracy'])

#### Mètriques sobre BT4SA_Subset

In [ ]:
#1 - Avaluació del model
SEResnet50_model3.evaluate(test_generator)

In [ ]:
#2 - Generació de prediccions
y_test_pred = SEResnet50_model3.predict(test_generator)
#3 - Classification report
y_test_pred_class = np.argmax(y_test_pred,axis=1)
class_report_resnet50 = classification_report(y_test_true, y_test_pred_class)
print(class_report_resnet50)

In [ ]:
#4 - Matriu de confusió
plot_confusion_matrix(y_test_true, y_test_pred_class, 'Model SE-ResNet-50 (Nadam) - BT4SA Dataset')

#### Mètriques sobre Twitter_Dataset

In [ ]:
#1 - Generació de prediccions
y_test_pred_tds = SEResnet50_model3.predict(test_generator_tds)
#2 - Classification report
y_test_pred_class_tds = np.argmax(y_test_pred_tds,axis=1)
#zero_division=1- el DS no te exemples de casos 'neutre'
class_report_seresnet50 = classification_report(y_tds_test_true, y_test_pred_class_tds, zero_division=1)
print(class_report_seresnet50)

In [ ]:
#3 - Matriu de confusió
plot_confusion_matrix(y_tds_test_true, y_test_pred_class_tds, 'Model SE-ResNet-50 (Nadam) - Twitter Dataset')

## Visualitzacions

### Input needed: Identificació de fitxers

Per tal de mostrar gràficament l'evolució de les diferents mètriques durant l'entrenament i com que aquest s'ha dut a terme en blocs degut a les restriccions sobre els recursos computacionals necessaris, es necessita crear una llista amb els fitxers a concatenar i que conformen un únic entrenament

In [ ]:
#Fitxers Resnet50

resnet50_adam_files = ['resnet50_fit_hist_adam_part1.npy', 'resnet50_fit_hist_adam_part2.npy']
resnet50_sgdnag_files = ['resnet50_fit_hist_SGD_NAG_part1.npy', 'resnet50_fit_hist_SGD_NAG_part2.npy']
resnet50_nadam_files = ['resnet50_fit_hist_nadam_part1.npy', 'resnet50_fit_hist_nadam_part2.npy']

#Fitxers SEResnet50

seresnet50_adam_files = ['seresnet50bm_fit_hist_adam_part1.npy', 'seresnet50bm_fit_hist_adam_part2.npy']
seresnet50_sgdnag_files = ['seresnet50_fit_hist_SGD_NAG_part1.npy', 'seresnet50_fit_hist_SGD_NAG_part2.npy']
seresnet50_nadam_files = ['seresnet50_fit_hist_nadam_part1.npy']

#Ruta completa 

resnet50_adam_files = [RESULTATS_DIR + file for file in resnet50_adam_files]
resnet50_sgdnag_files = [RESULTATS_DIR + file for file in resnet50_sgdnag_files]
resnet50_nadam_files = [RESULTATS_DIR + file for file in resnet50_nadam_files]


seresnet50_adam_files = [RESULTATS_DIR + file for file in seresnet50_adam_files]
seresnet50_sgdnag_files = [RESULTATS_DIR + file for file in seresnet50_sgdnag_files]
seresnet50_nadam_files = [RESULTATS_DIR + file for file in seresnet50_nadam_files]

In [ ]:
test = np.load(RESULTATS_DIR + 'resnet50_fit_hist_adam_part1.npy' ,allow_pickle=True).item()

In [ ]:
#Funció per concatenar els històrics d'entrenament executats en diferents sessions
def generatePlotData(mfit_list):
    params = { "loss" : [], "accuracy": [], "val_loss": [], "val_accuracy": [] }

    for mfit_hist in mfit_list:
        training_data = np.load(mfit_hist,allow_pickle=True).item()
        for metric in params:
            params[metric] = params[metric] + training_data[metric]

    return params


In [ ]:
#Generació de les dades concatenades
resnet50_adam_hist =  generatePlotData(resnet50_adam_files)
resnet50_sgdnag_hist =  generatePlotData(resnet50_sgdnag_files)
resnet50_nadam_hist =  generatePlotData(resnet50_nadam_files)

seresnet50_adam_hist =  generatePlotData(seresnet50_adam_files)
seresnet50_sgdnag_hist =  generatePlotData(seresnet50_sgdnag_files)
seresnet50_nadam_hist = generatePlotData(seresnet50_nadam_files)

### Entrenaments

In [ ]:
#Funció per mostrar les mètriques de l'entrenament d'un mòdel
def plot_metrics(mfit_hist, epochs, model_name):
    fig, axs = plt.subplots(1,2, figsize=(15,5))
    #axs[0] -> loss
    axs[0].plot(np.arange(0, epochs), mfit_hist["loss"], label="loss")
    axs[0].plot(np.arange(0, epochs), mfit_hist["val_loss"], label="val_loss")
    axs[0].set_title("Training loss -" + model_name)
    axs[0].set_xlabel('Epoch')
    axs[0].set_ylabel('Loss')
    axs[0].legend(loc="lower left")
    #axs[1] -> Accuracy
    axs[1].plot(np.arange(0, epochs), mfit_hist["accuracy"], label="accuracy")
    axs[1].plot(np.arange(0, epochs), mfit_hist["val_accuracy"], label="val_accuracy")
    axs[1].set_title("Training accuracy -" + model_name)
    axs[1].set_xlabel('Epoch')
    axs[1].set_ylabel('Accuracy')
    axs[1].legend(loc="upper left") 
    
#Funció per mostrar i comparar  mètriques entre diversos models 
colors = ['b','r', 'g', 'c', 'm', 'y']
def plot_nmodels_metrics(n_models_metrics):
    fig, axs = plt.subplots(2,1, figsize=(15,20))
    i = 0
    for model, metrics in n_models_metrics.items():
        #axs[0] -> loss
        axs[0].plot(np.arange(0, len(metrics["metrics"]["loss"])), metrics["metrics"]["loss"], label=(model + '(training)'), color=colors[i])
        axs[0].plot(np.arange(0, len(metrics["metrics"]["val_loss"])), metrics["metrics"]["val_loss"], label=(model + '(validation)'), color=colors[i], linestyle='--')
        axs[0].set_xlabel('Epoch')
        axs[0].set_ylabel('Loss')
        axs[0].legend(loc="lower left")
        #axs[1] -> accuracy
        axs[1].plot(np.arange(0, len(metrics["metrics"]["accuracy"])), metrics["metrics"]["accuracy"], label=(model + '(training)'), color=colors[i])
        axs[1].plot(np.arange(0, len(metrics["metrics"]["val_accuracy"])), metrics["metrics"]["val_accuracy"], label=(model + '(validation)'), color=colors[i], linestyle='--')
        axs[1].set_xlabel('Epoch')
        axs[1].set_ylabel('Accuracy')
        axs[1].legend(loc="upper left")
        i+=1

In [ ]:
plot_metrics(resnet50_adam_hist, 
             len(resnet50_adam_hist['loss']),
             "ResNet-50 (Adam Opt.)")
plot_metrics(resnet50_sgdnag_hist, 
             len(resnet50_sgdnag_hist['loss']),
             "ResNet-50 (SGD with Nesterov Opt.)")

plot_metrics(resnet50_nadam_hist, 
             len(resnet50_nadam_hist['loss']),
             "ResNet-50 (Nadam Opt.)")

plot_metrics(seresnet50_adam_hist, 
             len(seresnet50_adam_hist['loss']),
             "SEResNet-50 (Adam Opt.)")

plot_metrics(seresnet50_sgdnag_hist, 
             len(seresnet50_sgdnag_hist['loss']),
             "SE-ResNet-50 (SGD with Nesterov Opt.)")

plot_metrics(seresnet50_sgdnag_hist, 
             len(seresnet50_sgdnag_hist['loss']),
             "SE-ResNet-50 (SGD with Nesterov Opt.)")

plot_metrics(seresnet50_nadam_hist, 
             len(seresnet50_nadam_hist['loss']),
             "SE-ResNet-50 (Nadam Opt.)")


### Comparació de mètriques entre entrenaments per al mateix model

#### ResNet50

In [ ]:
#Preparem un diccionari amb les mètriques de tots els models a comparar
test_all_resnetmodels = {'Resnet-50 Adam Opt.' : {'metrics' : resnet50_adam_hist}, 'Resnet-50 SGD Opt.': {'metrics': resnet50_sgdnag_hist},
                         'Resnet-50 Nadam Opt.' : {'metrics' : resnet50_nadam_hist}}


plot_nmodels_metrics(test_all_resnetmodels)

#### SEResNet50

In [ ]:
#Preparem un diccionari amb les mètriques de tots els models a comparar
test_all_seresnetmodels = {'SE-ResNet-50 Adam Opt.' : {'metrics' : seresnet50_adam_hist}, 'SE-ResNet-50 SGD Opt.': {'metrics': seresnet50_sgdnag_hist},
                           'SE-ResNet-50 Nadam Opt.' : {'metrics' : seresnet50_nadam_hist}}


plot_nmodels_metrics(test_all_seresnetmodels)

### Comparació de mètriques entre tots els entrenaments dels dos models

In [ ]:
test_all_models = {'Resnet-50 Adam Opt.' : {'metrics' : resnet50_adam_hist}, 'Resnet-50 SGD Opt.': {'metrics': resnet50_sgdnag_hist}, 'Resnet-50 Nadam Opt.': {'metrics': resnet50_nadam_hist},
                   'SE-Resnet-50 Adam Opt.' : {'metrics' : seresnet50_adam_hist},  'SE-Resnet-50 SGD Opt.' : {'metrics' : seresnet50_sgdnag_hist}, 'SE-Resnet-50 Nadam Opt.' : {'metrics' : seresnet50_nadam_hist}}

In [ ]:
plot_nmodels_metrics(test_all_models)

### Visualitzacions sobre les imatges

In [ ]:
#Seleccio d'imatges
positive_files = ['test/positiu/769107137960239104-1.jpg', 'test/positiu/784505849788137472-1.jpg', 'test/positiu/781898506868887554-1.jpg']
neutral_files = ['test/neutre/769387036432330752-1.jpg','769509048718884864-1/769509048718884864-1.jpg','test/positiu/769549423101685761-1.jpg']
negative_files = ['test/negatiu/795805388650254337-2.jpg','test/negatiu/796064542128218112-4.jpg', 'test/negatiu/798394540910608384-2.jpg']

#### Visualització de filtres

In [ ]:
img_path = BT4SA_DS_DIR + positive_files[0]


Imatge original

In [ ]:
display(Image(img_path))

Visualitzem els primers filtres de cada model per poder adquirir una intuició les característiques extretes per cada model

In [ ]:
def viz_filters(activations):
    square = 8
    ix = 1
    for _ in range(square):
        for _ in range(square):
            # specify subplot and turn of axis
            ax = plt.subplot(square, square, ix)
            ax.set_xticks([])
            ax.set_yticks([])
            # plot filter channel in grayscale
            plt.imshow(activations[0, :, :, ix-1], cmap='gray')
            ix += 1
    # show the figure
    plt.show()    


**Resnet50**

In [ ]:
#Preprocessat de la imatge
img_tensor = resnet50_in_prep(img_to_array(load_img(img_path, target_size=(IM_HEIGHT,IM_WIDTH))))
img_tensor = np.expand_dims(img_tensor, axis=0)
#Sub-model que conté el primer fins al primer block convolucional 
layer_outputs = [layer.output for layer in resnet50_model2.layers[0:20]] 

activation_model = Model(inputs=resnet50_model2.input, outputs=layer_outputs) 

activations_resnet50 = activation_model.predict(img_tensor) 

In [ ]:
activation_model.summary()

Filtres al primer bloc convolucional

In [ ]:
conv_activation = activations_resnet50[2]
viz_filters(conv_activation)

Filtres despres del primer bloc residual

In [ ]:
conv_activation = activations_resnet50[-1]
viz_filters(conv_activation)

**SEResnet50**

In [ ]:
#Preprocessat de la imatge
img_tensor = seresnet50_in_prep(img_to_array(load_img(img_path, target_size=(IM_HEIGHT,IM_WIDTH))))
img_tensor = np.expand_dims(img_tensor, axis=0)
#Sub-model que conté el primer fins al primer block convolucional 
layer_outputs = [layer.output for layer in SEResnet50_model2.layers[0:28]] 

activation_model = Model(inputs=SEResnet50_model2.input, outputs=layer_outputs) 

activations_seresnet50 = activation_model.predict(img_tensor) 

In [ ]:
activation_model.summary()

Filtres al primer bloc convolucional

In [ ]:
viz_filters(activations_seresnet50[2])

Filtres despres del primer bloc Squeeze-and-excitation

In [ ]:
viz_filters(activations_seresnet50[-1])

In [ ]:
img_tensor = resnet50_in_prep(img_to_array(load_img(img_path, target_size=(IM_HEIGHT,IM_WIDTH))))
img_tensor = np.expand_dims(img_tensor, axis=0)
model = Model(inputs=resnet50_model.inputs, outputs=resnet50_model.layers[1].output)

feature_maps = model.predict(img_tensor)

#### Visualització dels Mapes d'activació: ResNet50 vs SEResNet50

In [ ]:
# Funcionalitat extreta de https://keras.io/examples/vision/grad_cam/
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, classifier_layer_names):
  # First, we create a model that maps the input image to the activations
  # of the last conv layer
  last_conv_layer = model.get_layer(last_conv_layer_name)
  last_conv_layer_model = Model(model.inputs, last_conv_layer.output)

  # Second, we create a model that maps the activations of the last conv
  # layer to the final class predictions
  classifier_input = Input(shape=last_conv_layer.output.shape[1:])
  x = classifier_input
  for layer_name in classifier_layer_names:
      x = model.get_layer(layer_name)(x)

  classifier_model = Model(classifier_input, x)

  # Then, we compute the gradient of the top predicted class for our input image
  # with respect to the activations of the last conv layer
  with tf.GradientTape() as tape:
    # Compute activations of the last conv layer and make the tape watch it
    last_conv_layer_output = last_conv_layer_model(img_array)
    tape.watch(last_conv_layer_output)
    # Compute class predictions
    preds = classifier_model(last_conv_layer_output)
    top_pred_index = tf.argmax(preds[0])
    top_class_channel = preds[:, top_pred_index]
  # This is the gradient of the top predicted class with regard to
  # the output feature map of the last conv layer
  grads = tape.gradient(top_class_channel, last_conv_layer_output)

  # This is a vector where each entry is the mean intensity of the gradient
  # over a specific feature map channel
  pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

  # We multiply each channel in the feature map array
  # by "how important this channel is" with regard to the top predicted class
  last_conv_layer_output = last_conv_layer_output.numpy()[0]
  pooled_grads = pooled_grads.numpy()
  for i in range(pooled_grads.shape[-1]):
    last_conv_layer_output[:, :, i] *= pooled_grads[i]

  # The channel-wise mean of the resulting feature map
  # is our heatmap of class activation
  heatmap = np.mean(last_conv_layer_output, axis=-1)

  # For visualization purpose, we will also normalize the heatmap between 0 & 1
  heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
  return heatmap

In [ ]:
#Funcion per visualitzar la imatge i el mapa d'activació 
def display_gradCAM(last_conv_layer_name, img_path, model, image_preprocessor, img_class):
    #Carreguem la imatge
    img = load_img(img_path, target_size=(IM_HEIGHT,IM_WIDTH))
    # Prepare image
    img_array = image_preprocessor(img_to_array(img))
    img_tensor = np.expand_dims(img, axis=0)
    img_class_pred = CLASSES_NAMES[np.argmax(model.predict(img_tensor))]

    test = img_to_array(img)
    #Identificació de les capes de classificació
    classifier_layer_names = [layer.name for layer in model.layers[-6:]]
    # Generació mapa d'activació
    heatmap = make_gradcam_heatmap(img_tensor, model, last_conv_layer_name, classifier_layer_names)
    # Superposició d'imatges
    # heatmap to a range 0-255
    heatmap_rs = np.uint8(255 * heatmap)
    # We use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # We use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap_rs]

    # We create an image with RGB colorized heatmap
    jet_heatmap = array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img_array.shape[1], img_array.shape[0]))
    jet_heatmap = img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * 0.5 + test
    superimposed_img = array_to_img(superimposed_img)
    
    # sub-plot per mostrar la terna: original, CAM, superimposed
    fig, axes = plt.subplots(1, 3, figsize=(20,5))
    axes[0].imshow(img) 
    axes[0].set_title('Predicció: ' + img_class_pred) 
    axes[1].imshow(heatmap_rs)
    axes[1].set_title('Grad-CAM')
    axes[1].xaxis.set_ticks([])
    axes[1].yaxis.set_ticks([])
    axes[2].imshow(superimposed_img)
    axes[2].set_title('superposició')
    fig.suptitle('Ground Truth - ' + img_class)
    plt.show()
    

In [ ]:
#Seleccio d'imatges
positive_files = ['test/positiu/769107137960239104-1.jpg', 'test/positiu/781948578457530368-4.jpg', 'test/positiu/781898506868887554-1.jpg']
neutral_files = ['test/neutre/769387036432330752-1.jpg','test/neutre/769509048718884864-1.jpg','test/neutre/769549423101685761-1.jpg']
negative_files = ['test/negatiu/795805388650254337-2.jpg','test/negatiu/796064542128218112-4.jpg', 'test/negatiu/798394540910608384-2.jpg']

In [ ]:
def display_multiple_gradCAM(file_list, class_type, last_conv_layer_name, model, preprocessor):
    for img_fileName in file_list:
         img_path = BT4SA_DS_DIR + img_fileName
         display_gradCAM(last_conv_layer_name, img_path, model, preprocessor, class_type)   
    

### ResNet50

In [ ]:
#Capes per visualitzar
last_conv_layer_name = "conv5_block3_out"

#Sentiment positiu 
display_multiple_gradCAM(positive_files, "Positiu", last_conv_layer_name,resnet50_model2, resnet50_in_prep)

In [ ]:
#Sentiment neutre
display_multiple_gradCAM(neutral_files, "Neutre", last_conv_layer_name,resnet50_model2, resnet50_in_prep)

In [ ]:
#Sentiment negatiu
display_multiple_gradCAM(negative_files, "Negatiu", last_conv_layer_name, resnet50_model2, resnet50_in_prep)

### SEResNet50

In [ ]:
last_conv_layer_name = "activation_80"

#Sentiment positiu 
display_multiple_gradCAM(positive_files, "Positiu", last_conv_layer_name, SEResnet50_model1, seresnet50_in_prep);

In [ ]:
#Sentiment Neutre 
display_multiple_gradCAM(neutral_files, "Neutre", last_conv_layer_name, SEResnet50_model1, seresnet50_in_prep)

In [ ]:
#Sentiment Negatiu 
display_multiple_gradCAM(negative_files, "Negatiu", last_conv_layer_name, SEResnet50_model1, seresnet50_in_prep)